In [17]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import codecs
import json
import os
import pickle
import re
import sys
import time
import traceback

import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from glob import glob
from os.path import splitext, split, join, exists
from tqdm import tqdm
from IPython import display

In [18]:
from model_tools.gentools import gentools as gntls
from model_tools.gentools import advtools as advtls
from model_tools.plot import feature_plot as fplt
from model_tools.feature_select import feature_select as fs
from model_tools.feature_proc import feature_proc as fp
from model_tools.model import model as mdl

### 特征筛选

In [19]:
target = 'target'
dpath = 'E:/git/test_data/'
cfg_path = 'E:/git/model_tools/notebook_demo/'
exclude_cols = ['OBJECTNO',
 'target',
 'PUTOUTSERIALNO',
 'CUSTOMERID',
 'order_create_time',
 'order_create_at',
 'SERIALNO',
 'NATURALOCCURDATE',
 'APPLSERIALNO',
 'ruleSERIALNO',
 'LK_MOBILENO',
 'RULESERIALNO',
 'JIAOYIRULE_SERIALNO',
 'ref_time']

In [20]:
x_train = pd.read_pickle(join(dpath, 'x_train.pkl'))
x_test = pd.read_pickle(join(dpath, 'x_test.pkl'))
oot = pd.read_pickle(join(dpath, 'oot.pkl'))
print(x_train.shape,x_test.shape,oot.shape)
print(x_train[target].mean(),x_test[target].mean(),oot[target].mean())
x_train.head(2)

(70224, 1141) (30096, 1141) (34251, 1141)
0.059139325586694005 0.05911084529505582 0.053166330910046425


,OBJECTNO,target,PUTOUTSERIALNO,CUSTOMERID,order_create_time,order_create_at,SERIALNO,NATURALOCCURDATE,APPLSERIALNO,ruleSERIALNO,...,LK_EDUCATION_proc,LK_V_RANGE_proc,DENGJI_proc,LK_FACEVERIFYINFO_pairSimilarity1,LK_FACEVERIFYINFO_pairSimilarity2,province,地区,级别,城市等级,province_
0,1020042300003186,0.0,1020042300003472,1020020300010134,2020-04-23 09:05:56,1587603956,1020042300003472,2020-04-23,1020020300002636,1020020300003028,...,NaN,3.0,3,NaN,NaN,上海,上海,特级城市,1.0,14
1,1020032300002268,0.0,1020032300000295,1020020300010134,2020-03-23 08:47:56,1584924476,1020032300000295,2020-03-23,1020020300002636,1020020300003028,...,NaN,3.0,3,NaN,NaN,上海,上海,特级城市,1.0,14


### dropna

In [21]:
is_flag = False
thresh = 0.98

In [22]:
x_train, drop_cols = fs.drop_na_cols(x_train, thresh=thresh, cols=None, 
                                     type_='calc', is_flag=is_flag, exclude=exclude_cols)
x_test, _ = fs.drop_na_cols(x_test, thresh=thresh, cols=drop_cols,
                      type_='nocalc', is_flag=is_flag, exclude=exclude_cols)
oot, _ = fs.drop_na_cols(oot, thresh=thresh, cols=drop_cols,
                      type_='nocalc', is_flag=is_flag, exclude=exclude_cols)

print(x_train.shape, x_test.shape, oot.shape)
print(x_train[target].mean(),x_test[target].mean(),oot[target].mean())

# of columns to drop: 0
(70224, 1141) (30096, 1141) (34251, 1141)
0.059139325586694005 0.05911084529505582 0.053166330910046425


In [23]:
is_flag = True
thresh = 0.95

In [24]:
x_train, drop_cols = fs.drop_na_cols(x_train, thresh=thresh, cols=None, 
                                     type_='calc', is_flag=is_flag, exclude=exclude_cols)
x_test, _ = fs.drop_na_cols(x_test, thresh=thresh, cols=drop_cols,
                      type_='nocalc', is_flag=is_flag, exclude=exclude_cols)
oot, _ = fs.drop_na_cols(oot, thresh=thresh, cols=drop_cols,
                      type_='nocalc', is_flag=is_flag, exclude=exclude_cols)

print(x_train.shape, x_test.shape, oot.shape)
print(x_train[target].mean(),x_test[target].mean(),oot[target].mean())

0it [00:00, ?it/s]

# of columns to drop: 0
# start add is_na flag



0it [00:00, ?it/s]
0it [00:00, ?it/s]

# start add is_na flag
# start add is_na flag


(70224, 1141) (30096, 1141) (34251, 1141)
0.059139325586694005 0.05911084529505582 0.053166330910046425


### drop single value

In [25]:
is_flag = False
thresh = 0.98

In [26]:
x_train, cols_inform = fs.drop_singleValue_cols(x_train, thresh=thresh, cols=None, 
                                                type_='calc', is_flag=is_flag, exclude=exclude_cols)
x_test, _ = fs.drop_singleValue_cols(x_test, thresh=thresh, cols=cols_inform,
                               type_='nocalc', is_flag=is_flag, exclude=exclude_cols, save_file=None)
oot, _ = fs.drop_singleValue_cols(oot, thresh=thresh, cols=cols_inform,
                               type_='nocalc', is_flag=is_flag, exclude=exclude_cols, save_file=None)
print(x_train.shape, x_test.shape, oot.shape)
print(x_train[target].mean(),x_test[target].mean(),oot[target].mean())

100%|█████████████████████████████████████████████████████████████████████████████| 1127/1127 [00:06<00:00, 174.57it/s]


(70224, 1141) (30096, 1141) (34251, 1141)
0.059139325586694005 0.05911084529505582 0.053166330910046425


In [27]:
is_flag = True
thresh = 0.95

In [28]:
x_train, cols_inform = fs.drop_singleValue_cols(x_train, thresh=thresh, cols=None, 
                                                type_='calc', is_flag=is_flag, exclude=exclude_cols, 
                                                save_file=join(cfg_path,'cfg','SingleValuecfg'))

 10%|████████▎                                                                          | 6/60 [00:00<00:01, 51.05it/s]

# start add is_mode flag


100%|██████████████████████████████████████████████████████████████████████████████████| 60/60 [00:02<00:00, 24.73it/s]


In [29]:
# with open(join(cfg_path,'cfg','SingleValuecfg'),'r') as load_f:
#     cols_inform = json.load(load_f)

In [30]:
x_test, _ = fs.drop_singleValue_cols(x_test, thresh=thresh, cols=cols_inform,
                               type_='nocalc', is_flag=is_flag, exclude=exclude_cols, save_file=None)
oot, _ = fs.drop_singleValue_cols(oot, thresh=thresh, cols=cols_inform,
                               type_='nocalc', is_flag=is_flag, exclude=exclude_cols, save_file=None)
print(x_train.shape, x_test.shape, oot.shape)
print(x_train[target].mean(),x_test[target].mean(),oot[target].mean())

 13%|███████████                                                                        | 8/60 [00:00<00:00, 76.39it/s]

# start add is_mode flag


 13%|███████████                                                                        | 8/60 [00:00<00:00, 74.80it/s]

# start add is_mode flag


100%|██████████████████████████████████████████████████████████████████████████████████| 60/60 [00:01<00:00, 47.79it/s]


(70224, 1141) (30096, 1141) (34251, 1141)
0.059139325586694005 0.05911084529505582 0.053166330910046425


In [31]:
feas = 'ovd_gt1_repay_amt_rf_repay_max_p66d_add_mode_flag'

tab = fplt.TabOut()
params = {}
for v in ['x_train', 'x_test', 'oot']:
    params.update({v: fplt.PlotUtils().plot_bivar(eval(v), feas=feas, target=target,
                    cut_params=None, title=feas,
                    yaxis='count_rto', pyecharts=False, mark_line=True,
                    draw_lin=False, save_path=None, thred=3, max_bins=3,
                    color_bar=['steelblue'], color_line=['red','black'], return_state=False)})
    
tab.get_tab_out(params, box_style='info',column_cnt=5)

VBox(box_style='info', children=(HBox(children=(HBox(children=(Button(description='x_train', layout=Layout(hei…

### select by filter

#### calc filter_index

In [20]:
feas_cut, feas_nocut = [],[]
for fea in tqdm(list(set(x_train.select_dtypes(np.number).columns)-set(exclude_cols))):
    if x_train[fea].nunique()<=20:
        feas_nocut.append(fea)
    else:
        feas_cut.append(fea)
print(len(feas_nocut), len(feas_cut))

100%|█████████████████████████████████████████████████████████████████████████████| 1086/1086 [00:02<00:00, 540.66it/s]

174 912


* 加上mic会比较慢

In [21]:
filter_index1,_ = fs.calc_feas_filterindex(x_train, feas_list=feas_nocut, target=target, 
                                         calc_list='all', cut=None, drop_value=None, exclude_feas=exclude_cols)
filter_index2,_ = fs.calc_feas_filterindex(x_train, feas_list=feas_cut, target=target, 
                                         calc_list='all', cut=10, drop_value=None, exclude_feas=exclude_cols)
filter_index = pd.concat([filter_index2,filter_index1])
print(filter_index.shape)
filter_index.head()

  0%|                                                                                          | 0/912 [00:00<?, ?it/s]

# of feas cnt not cut: 0


100%|████████████████████████████████████████████████████████████████████████████████| 912/912 [32:30<00:00,  2.14s/it]

# of feas cnt not cut: 0
(1086, 10)


,Person,Fclassif,Chi2,inform_gain,inform_gain_rto,IV,absWOE,AUC,KS,MIC
ely_gt0_repay_amt_rf_repay_mean_p186d,-0.070881,349.018348,465.597633,0.004831,0.001697,0.123534,2.199691,0.5898,0.148264,0.004831
ely_gt3_repay_amt_rf_repay_sum_p36d,-0.078920,419.824320,417.902770,0.005597,0.004155,0.165306,2.348694,0.5757,0.139082,0.005597
ely_gt30_repay_oid_cnt_rf_repay_p186d,-0.079474,439.342060,452.816829,0.005815,0.005138,0.169808,2.767007,0.5731,0.142316,0.005815
normal_days_rf_due_max_freq_p96d,-0.068264,319.221427,324.722007,0.004039,0.002229,0.114639,2.353433,0.5725,0.123482,0.004039
ovd_gt1_repay_amt_rf_due_sum_p66d,0.095386,625.907014,620.230444,0.004148,0.027702,0.089157,1.512713,0.5295,0.058938,0.004148


In [22]:
filter_index.to_csv(join(cfg_path,'cfg','filter_index.csv'))

* 不加mic快很多

In [23]:
# filter_index1,_ = fs.calc_feas_filterindex(x_train, feas_list=feas_nocut, target=target, 
#                                          calc_list=['CalcPerson', 'CalcFclassif', 'CalcChi2', 'CalcInformGain', 'CalcIV', 'CalcAucKs'], 
#                                            cut=None, drop_value=None, exclude_feas=exclude_cols)
# filter_index2,_ = fs.calc_feas_filterindex(x_train, feas_list=feas_cut, target=target, 
#                                          calc_list=['CalcPerson', 'CalcFclassif', 'CalcChi2', 'CalcInformGain', 'CalcIV', 'CalcAucKs'], 
#                                            cut=10, drop_value=None, exclude_feas=exclude_cols)
# filter_index = pd.concat([filter_index2,filter_index1])
# print(filter_index.shape)
# filter_index.head()

#### feas sel by filyer_index

In [5]:
filter_index= pd.read_csv(join(cfg_path,'cfg','filter_index.csv'),index_col=0)
print(filter_index.shape)
filter_index.head(2)

(1086, 10)


,Person,Fclassif,Chi2,inform_gain,inform_gain_rto,IV,absWOE,AUC,KS,MIC
ely_gt0_repay_amt_rf_repay_mean_p186d,-0.070881,349.018348,465.597633,0.004831,0.001697,0.123534,2.199691,0.5898,0.148264,0.004831
ely_gt3_repay_amt_rf_repay_sum_p36d,-0.078920,419.824320,417.902770,0.005597,0.004155,0.165306,2.348694,0.5757,0.139082,0.005597


In [32]:
sel_dict = {'Person':{'sorted':abs, 'best_number':80},
            'Fclassif':{'best_number':150},
            'Chi2': {'best_number':80},
            'inform_gain': {'best_number':100},
            'inform_gain_rto': {'best_number':50},
            'IV': {'best_number':100},
            'absWOE': {'best_number':150},
            'AUC': {'best_number':150},
            'KS': {'best_number':150},
            'MIC':{'best_number':100}
           }

In [33]:
feas_sel = []
for key, value in sel_dict.items():
    sorted_value = value.get('sorted', None)
    number = value.get('best_number', 100)
    feas_ = gntls.SelectKBest(filter_index[key], SelectKBest=number, thred=-1, key=sorted_value)
    feas_sel.extend(feas_)
feas_sel = list(set(feas_sel))
len(feas_sel)

457

In [29]:
tab = fplt.TabOut()
params = {}

for fea in filter_index.sort_values('Person',ascending=False, key=abs).index[:5]:
    params.update({fea: fplt.PlotUtils().plot_bivar(x_train, feas=fea, target=target,
                    cut_params=None, title=fea,
                    yaxis='count_rto', pyecharts=False, mark_line=True,
                    draw_lin=False, save_path=None, thred=1, max_bins=999,
                    color_bar=['steelblue'], color_line=['red','black'], return_state=False)})
    
tab.get_tab_out(params, box_style='info',column_cnt=5)

VBox(box_style='info', children=(HBox(children=(HBox(children=(Button(description='ovd_days_rf_repay_sum_p7d',…

In [30]:
tab = fplt.TabOut()
params = {}

for fea in filter_index.sort_values('Fclassif',ascending=False).index[:5]:
    params.update({fea: fplt.PlotUtils().plot_bivar(x_train, feas=fea, target=target,
                    cut_params=None, title=fea,
                    yaxis='count_rto', pyecharts=False, mark_line=True,
                    draw_lin=False, save_path=None, thred=1, max_bins=999,
                    color_bar=['steelblue'], color_line=['red','black'], return_state=False)})
    
tab.get_tab_out(params, box_style='info',column_cnt=5)

VBox(box_style='info', children=(HBox(children=(HBox(children=(Button(description='ovd_days_rf_due_sum_p36d', …

In [31]:
tab = fplt.TabOut()
params = {}

for fea in filter_index.sort_values('Chi2',ascending=False).index[:5]:
    params.update({fea: fplt.PlotUtils().plot_bivar(x_train, feas=fea, target=target,
                    cut_params=None, title=fea,
                    yaxis='count_rto', pyecharts=False, mark_line=True,
                    draw_lin=False, save_path=None, thred=1, max_bins=999,
                    color_bar=['steelblue'], color_line=['red','black'], return_state=False)})
    
tab.get_tab_out(params, box_style='info',column_cnt=5)

VBox(box_style='info', children=(HBox(children=(HBox(children=(Button(description='ovd_days_rf_due_median_p36d…

In [32]:
tab = fplt.TabOut()
params = {}

for fea in filter_index.sort_values('inform_gain',ascending=False).index[:5]:
    params.update({fea: fplt.PlotUtils().plot_bivar(x_train, feas=fea, target=target,
                    cut_params=10, title=fea,
                    yaxis='count_rto', pyecharts=False, mark_line=True,
                    draw_lin=False, save_path=None, thred=1, max_bins=999,
                    color_bar=['steelblue'], color_line=['red','black'], return_state=False)})
    
tab.get_tab_out(params, box_style='info',column_cnt=5)

VBox(box_style='info', children=(HBox(children=(HBox(children=(Button(description='ovd_days_rf_repay_median_p7…

In [33]:
tab = fplt.TabOut()
params = {}

for fea in filter_index.sort_values('inform_gain_rto',ascending=False).index[:5]:
    params.update({fea: fplt.PlotUtils().plot_bivar(x_train, feas=fea, target=target,
                    cut_params=None, title=fea,
                    yaxis='count_rto', pyecharts=False, mark_line=True,
                    draw_lin=False, save_path=None, thred=1, max_bins=999,
                    color_bar=['steelblue'], color_line=['red','black'], return_state=False)})
    
tab.get_tab_out(params, box_style='info',column_cnt=5)

VBox(box_style='info', children=(HBox(children=(HBox(children=(Button(description='ovd_gt3_repay_amt_rf_repay_…

In [34]:
tab = fplt.TabOut()
params = {}

for fea in filter_index.sort_values('IV',ascending=False).index[:5]:
    params.update({fea: fplt.PlotUtils().plot_bivar(x_train, feas=fea, target=target,
                    cut_params=10, title=fea,
                    yaxis='count_rto', pyecharts=False, mark_line=True,
                    draw_lin=False, save_path=None, thred=1, max_bins=999,
                    color_bar=['steelblue'], color_line=['red','black'], return_state=False)})
    
tab.get_tab_out(params, box_style='info',column_cnt=5)

VBox(box_style='info', children=(HBox(children=(HBox(children=(Button(description='normal_repay_amt_rf_repay_m…

In [35]:
tab = fplt.TabOut()
params = {}

for fea in filter_index.sort_values('absWOE',ascending=False).index[:5]:
    params.update({fea: fplt.PlotUtils().plot_bivar(x_train, feas=fea, target=target,
                    cut_params=None, title=fea,
                    yaxis='count_rto', pyecharts=False, mark_line=True,
                    draw_lin=False, save_path=None, thred=1, max_bins=999,
                    color_bar=['steelblue'], color_line=['red','black'], return_state=False)})
    
tab.get_tab_out(params, box_style='info',column_cnt=5)

VBox(box_style='info', children=(HBox(children=(HBox(children=(Button(description='ovd_gt1_repay_cnt_rf_repay_…

In [36]:
tab = fplt.TabOut()
params = {}

for fea in filter_index.sort_values('AUC',ascending=False).index[:5]:
    params.update({fea: fplt.PlotUtils().plot_bivar(x_train, feas=fea, target=target,
                    cut_params=10, title=fea,
                    yaxis='count_rto', pyecharts=False, mark_line=True,
                    draw_lin=False, save_path=None, thred=1, max_bins=999,
                    color_bar=['steelblue'], color_line=['red','black'], return_state=False)})
    
tab.get_tab_out(params, box_style='info',column_cnt=5)

VBox(box_style='info', children=(HBox(children=(HBox(children=(Button(description='normal_repay_amt_rf_repay_s…

In [37]:
tab = fplt.TabOut()
params = {}

for fea in filter_index.sort_values('KS',ascending=False).index[:5]:
    params.update({fea: fplt.PlotUtils().plot_bivar(x_train, feas=fea, target=target,
                    cut_params=10, title=fea,
                    yaxis='count_rto', pyecharts=False, mark_line=True,
                    draw_lin=False, save_path=None, thred=1, max_bins=999,
                    color_bar=['steelblue'], color_line=['red','black'], return_state=False)})
    
tab.get_tab_out(params, box_style='info',column_cnt=5)

VBox(box_style='info', children=(HBox(children=(HBox(children=(Button(description='normal_repay_amt_rf_repay_s…

In [38]:
tab = fplt.TabOut()
params = {}

for fea in filter_index.sort_values('MIC',ascending=False).index[:5]:
    params.update({fea: fplt.PlotUtils().plot_bivar(x_train, feas=fea, target=target,
                    cut_params=10, title=fea,
                    yaxis='count_rto', pyecharts=False, mark_line=True,
                    draw_lin=False, save_path=None, thred=1, max_bins=999,
                    color_bar=['steelblue'], color_line=['red','black'], return_state=False)})
    
tab.get_tab_out(params, box_style='info',column_cnt=5)

VBox(box_style='info', children=(HBox(children=(HBox(children=(Button(description='ovd_days_rf_repay_median_p7…

### plot feas

In [ ]:
matplotlib.use("Agg")

In [41]:
print(len(feas_nocut), len(feas_cut))
feas_nocut = list(set(feas_nocut).intersection(set(feas_sel)))
feas_cut = list(set(feas_cut).intersection(set(feas_sel)))
print(len(feas_nocut), len(feas_cut))

174 912
100 357


In [42]:
for fea in tqdm(feas_nocut+list(x_train.select_dtypes('object').columns)):
    pic= fplt.PlotUtils().plot_bivar(x_train, feas=fea, target=target, cut_params=None, title=fea,
                    yaxis='count_rto', pyecharts=False, mark_line=True,
                    draw_lin=False, save_path=join(cfg_path,'bivar'), 
                    color_bar=['steelblue'], color_line=['red','black'])
    
for fea in tqdm(feas_cut):
    pic = fplt.PlotUtils().plot_bivar(x_train, feas=fea, target=target, cut_params=10, title=fea,
                    yaxis='count_rto', pyecharts=False, mark_line=True,
                    draw_lin=False, save_path=join(cfg_path,'bivar'), 
                    color_bar=['steelblue'], color_line=['red','black'])        

100%|████████████████████████████████████████████████████████████████████████████████| 357/357 [03:04<00:00,  1.94it/s]


### save

In [35]:
cat_feas = list(x_train.select_dtypes('object'))
x_train[list(set(feas_sel+exclude_cols+[target]+cat_feas))].to_pickle(join(dpath,'x_train_afsel.pkl'))
x_test[list(set(feas_sel+exclude_cols+[target]+cat_feas))].to_pickle(join(dpath,'x_test_afsel.pkl'))
oot[list(set(feas_sel+exclude_cols+[target]+cat_feas))].to_pickle(join(dpath,'oot_afsel.pkl'))